In [1]:
from neo4j import GraphDatabase, basic_auth

driver = GraphDatabase.driver(
    "bolt://3.235.199.126:7687", 
    auth=basic_auth("neo4j", "adverb-steel-lieutenant"))
session = driver.session()

In [2]:
my_graph = {"persons":
 [{"name":"Jack Kerouac", "year of birth":"1922", "year of death":"1969", "studied at":"Columbia University"}, 
  {"name":"Allen Ginsberg", "year of birth":"1926", "year of death":"1997", "studied at":"Columbia University"},
  {"name":"William S. Burroughs", "year of birth":"1914", "year of death":"1997", "studied at":"Harvard University"},
  {"name":"Neal Cassady", "year of birth":"1926", "year of death":"1968", "studied at":""},
  {"name":"Gary Snyder", "year of birth":"1930", "year of death":"", "studied at":"Reed College"},
  {"name":"Carolyn Cassady", "year of birth":"1923", "year of death":"2013", "studied at":""},
  {"name":"Lucien Carr", "year of birth":"1925", "year of death":"2005", "studied at":"Columbia University"},
  {"name":"Peter Orlovsky", "year of birth":"1933", "year of death":"2010", "studied at":"Columbia University"},
  {"name":"Philip Whalen", "year of birth":"1923", "year of death":"2002", "studied at":"Reed College"},
  {"name":"John Clellon Holmes", "year of birth":"1926", "year of death":"1988", "studied at":""}
 ],
 "books":
 [{"name":"On the Road", "year of publication":"1957"},
  {"name":"The Dharma Bums", "year of publication":"1958"},
  {"name":"Big Sur", "year of publication":"1962"},
  {"name":"Desolation Angels", "year of publication":"1965"}
 ],
"colleges":
 [{"name":"Columbia University", "city":"New York"},
  {"name":"Harvard University", "city":"Cambridge"},
  {"name":"Reed College", "city":"Portland"},
 ],
"characters":
 [{"name":"Sal Paradise", "book":"On the Road", "alter ego": "Jack Kerouac"},
  {"name":"Carlo Marx", "book":"On the Road", "alter ego": "Allen Ginsberg"},
  {"name":"Old Bull Lee", "book":"On the Road", "alter ego": "William S. Burroughs"},
  {"name":"Dean Moriarty", "book":"On the Road", "alter ego": "Neal Cassady"},
  {"name":"Camille", "book":"On the Road", "alter ego": "Carolyn Cassady"},
  {"name":"Damion", "book":"On the Road", "alter ego": "Lucien Carr"},
  {"name":"Ian MacArthur", "book":"On the Road", "alter ego": "John Clellon Holmes"},
  
  {"name":"Ray Smith", "book":"The Dharma Bums", "alter ego": "Jack Kerouac"},
  {"name":"Alvah Goldbook", "book":"The Dharma Bums", "alter ego": "Allen Ginsberg"},
  {"name":"Cody Pomeray", "book":"The Dharma Bums", "book2":"Big Sur", "book3":"Desolation Angels", "alter ego": "Neal Cassady"},
  {"name":"Japhy Ryder", "book":"The Dharma Bums", "alter ego": "Gary Snyder"},
  {"name":"Evelyn", "book":"The Dharma Bums", "book2":"Big Sur", "book3":"Desolation Angels", "alter ego": "Carolyn Cassady"},
  {"name":"George", "book":"The Dharma Bums", "alter ego": "Peter Orlovsky"},
  {"name":"Warren Coughlin", "book":"The Dharma Bums", "alter ego": "Philip Whalen"},
  
  {"name":"Jack Duluoz", "book":"Big Sur", "book2":"Desolation Angels", "alter ego": "Jack Kerouac"},
  {"name":"Irwin Garden", "book":"Big Sur", "book2":"Desolation Angels", "alter ego": "Allen Ginsberg"},
  {"name":"Jarry Wagner", "book":"Big Sur", "book2":"Desolation Angels", "alter ego": "Gary Snyder"},
  {"name":"Ben Fagan", "book":"Big Sur", "alter ego": "Philip Whalen"},
  {"name":"Julien", "book":"Big Sur", "alter ego": "Lucien Carr"},
  
  {"name":"Bull Hubbard", "book":"Desolation Angels", "alter ego": "William S. Burroughs"},
  {"name":"Julien Love", "book":"Desolation Angels", "alter ego": "Lucien Carr"},
  {"name":"Simon Darlovsky", "book":"Desolation Angels", "alter ego": "Peter Orlovsky"}
 ]
}

In [3]:
def create_node(label, features):
    req="CREATE (:"+label+" {"+features+"})"
    return session.run(req)
def update_node(name, feat_name, feat):
    req="MATCH (n {name:'"+name+"'}) SET n."+feat_name+" = '"+feat+"'"
    return session.run(req)

In [4]:
for i, pers in enumerate(my_graph["persons"]):
    label = 'person'
    features = f'name:"{pers["name"]}", birth_year:"{pers["year of birth"]}", death_year:"{pers["year of death"]}", studied:"{pers["studied at"]}"'
    create_node(label, features)
for i, book in enumerate(my_graph["books"]):
    label = 'book'
    features = f'name:"{book["name"]}", public_year:"{book["year of publication"]}"'
    create_node(label, features)
for i, college in enumerate(my_graph["colleges"]):
    label = 'college'
    features = f'name:"{college["name"]}", city:"{college["city"]}"'
    create_node(label, features)
for i, char in enumerate(my_graph["characters"]):
    label = 'character'
    features = f'name:"{char["name"]}", book:"{char["book"]}", alterego:"{char["alter ego"]}"'
    create_node(label, features)

In [5]:
for i, char in enumerate(my_graph["characters"]):
    if "book2" in char.keys():
        update_node(char["name"], "book2", char["book2"])
    if "book3" in char.keys():
        update_node(char["name"], "book3", char["book3"])

In [6]:
session.run('match (b:book), (p:person) where p.name = "Jack Kerouac" create (p)-[r:IS_AUTHOR_OF]->(b)')

In [7]:
session.run('match (c:character), (p:person) where p.name = c.alterego create (c)-[r:IS_ALTER_EGO_OF]->(p)')

In [8]:
session.run('''match (c:character), (b:book) where c.book = b.name or c.book2 = b.name 
or c.book3 = b.name create (c)-[r:IS_CHARACTER_OF]->(b)''')

In [9]:
session.run('match (a), (b) where a.name = "Allen Ginsberg" and b.name = "Peter Orlovsky" create (b)-[r:IS_PARTNER_OF]->(a)')

In [10]:
session.run('''match (a), (b) where a.name = "Carolyn Cassady" and b.name = "Neal Cassady" 
create (a)-[r:IS_PARTNER_OF {since:1948, divorced:1963}]->(b)''')

In [11]:
session.run('match (p:person), (c:college) where p.studied=c.name create (p)-[r:STUDIED_AT]->(c)')

In [12]:
#session.run('match(n) detach delete n')